                    Aula: Classificações de intenções de IA

-> langchain: O langchain é um framework (uma estrutura de trabalho) que facilita a criação de aplicações complexas com modelos de linguagem, como o gemini. Pense nele como uma "caixa de ferramentas" para desenvolvedores que
querem construir algo mais sofisticado do que apenas uma simples chamada
á API.
   O langchain permite que você "encadeie" (dai o nome "Language Chain")
   diferentes componentes para construir uma aplicação completa. Ele pode
   fazer coisas como:

   -> Chamar diferentes modelos de linguagem: Ele tem conectores para o
   Gemini, OpenAi, e outros. Isso permite que você troque de modelo
   facilmente.

   -> Conectar modelos a fontes de dados: O LangChain permite que o seu
   modelo de linguagem acesse informações externas, como documentos, bases
   de dados ou até mesmo na internet. Isso é o que permite que ele responda
   perguntas sobre informações que ele não tem "na memória".

   -> Integrar com outras ferramentas: Ele pode se conectar a ferramentas
   como calculadoras, calendários ou APIs de busca, permitindo que o modelo
   execute ações no mundo real.

   -> Gerenciar Conversas: O LangChain ajuda a manter o "contexto" de uma
   conversa com um chatbot, para que ele possa se lembrar do que foi dito
   nas mensagens anteriores.
   
   Em resumo, se a API do Gemini é o "motor" da sua aplicação, o LangChain
   é o "chassi" e a "transmissão" que permite que você construa o carro
   completo, com todos os recursos que você precisa.

-> from langchain_google_genai: Esta é a biblioteca que conecta o langchain aos
modelos de IA generativa do Google (como o gemini). Ela contém todas as ferra
mentas necessárias para interagir com a API do Gemini.

-> import ChatGoogleGenerativeAI: Esta é a classe especifica que você precisa
para criar uma instância do modelo. Ela atua como um "conector" ou "adaptador"
entre a estrutura do langchain e a API do gemini. Quando você cria um objeto
a partir desta classe (hatGoogleGenerativeAI), ele se torna a sua ponte de
comunicação com o modelo de linguagem do Google.



In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

Acessando a chave da API na variável de ambiente

In [2]:
# Import da biblioteca que tem como objetivo acessar variáveis de ambiente
# com o objetivo de acessar a chave da nossa API.
import os

# Import da função que irá carregar as variáveis ambientes    
from dotenv import load_dotenv

# Carregando a variável de ambiente no código 
load_dotenv()

# função da biblioteca que irá acessar o valor da nossa variável
# de ambiente (a chave da nossa API). A função irá receber como
# argumento o nome da variável que irá conter a chave da API.*
api_key = os.getenv('GOOGLE_API_KEY')

# Essa estrutura condicional irá verificar se a chave foi carregada
# ou não no código
if api_key:
    
    # Se a condição for verdadeira, iremos apresentar essa mensagem
    print("A chave foi carregada")

else:
    
    # Se a condição for falsa, iremos apresentar essa mensagem
    print("A chave não foi carregada")

A chave foi carregada


Criando a classe do modelo de IA

In [3]:
# Instância da classe que irá criar o modelo do gemini
# que iremos utilizar na construção do agente. A classe
# irá receber em seu construtor os seguintes argumentos:
llm = ChatGoogleGenerativeAI(
    
    # O modelo de IA que será utilizado
    model = "gemini-2.5-flash",
    
    # A temperatura que irá definir o nivel de criatividade
    # do modelo utilizado.
    temperature = 0,
    
    # A chave que irá nos conectar a API do modelo escolhido
    api_key = api_key
    
    

)

Criando a nossa primeira interação com o modelo

In [4]:
# Irá conter a instrução que o usuário passará para o modelo
pergunta = str(input("Faça uma pergunta: "))

# Ira conter a resposta do modelo utilizando a função 
# invoke que tem como objetivo coletar a instrução do
# usuário (variável "pergunta") e criar uma resposta
resposta = llm.invoke(pergunta)

# Impressão da resposta utilizando o atributo content que
# irá conter apenas a resposta do modelo. Dessa maneira,
# a saida nao irá mostrar os outros atributos da classe.
print(resposta.content)

Olá! Tudo bem?


Aprimorando a nossa interação com o modelo: Resolvi fazer algo um pouco diferente da aula e criei um loop infinito que possibilita que o usuário
consiga ter uma conversa com o modelo.

In [5]:
# Ira criar o loop while infinito
while True:
    
    # Ira conter a instrução do usuário. O upper irá converter
    # todas as letras da instrução em maiusculas, dessa maneira
    # conseguiremos validar as instruções independente da maneira
    # que o usuário as digitar.
    pergunta = str(input("Faça uma pergunta: ")).upper()

    # Irá conter a resposta do modelo.
    resposta = llm.invoke(pergunta)

    # Irá imprimir a resposta do modelo.
    print(resposta.content)
    
    # Ira verificar se o usuário digitou a palavra "TCHAU"
    if pergunta == "TCHAU":
        
        # Se a condição for verdadeira iremos imprimir uma mensagem
        # e encerrar a sessão do usuário (usando a função break)
        print("Sessão Encerrada")
        
        break

Tchau!
Sessão Encerrada


Agora, iremos definir o prompt que irá ditar o comportamento do nosso modelo
de agente que irá analisar a solicitação e classifica-la em grupos das seguintes categorias:

grupo decisão: "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO"

grupo urgencia:  "BAIXA" | "MEDIA" | "ALTA"

grupo campos faltantes: Irá solicitar ao usuário que ele passe mais informações

In [6]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

-> Esse trecho de código usa a biblioteca Pydantic para criar um "molde" ou
"blueprint" para a estrutura de dados que você quer que o modelo do Gemini
retorne.

-> Ele garante que a resposta do modelo tenha exatamente o formato JSON
que você definiu no seu TRIAGEM_PROMPT, evitando erros e tornando a resposta
fácil de ser processada pelo seu código Python.

In [7]:
# Importa as classes BaseModel e Field da biblioteca pydantic
from pydantic import BaseModel, Field

# Biblioteca  que irá importar os tipos de dados necessários para
# a criação da estrutura do modelo.
# Literal: Serve para limitar um campo a um conjunto especifico de valores.
# Por exemplo, Literal ["BAIXA", "MEDIA", "ALTA"] garante que a "urgencia"
# só pode ter um desses três valores.

# List: Indica que o campo será uma lista.

# 
from typing import Literal, List, Dict

# Ira criar a nossa classe de modelo que urá herdar a classe BaseModel
class TriagemOut(BaseModel):
    
    # Define o campo "decisão" com base nos valores que especificamos
    # no prompt.
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    
    # Define o campo "urgencia" com base nos valores que especificamos
    # no prompt.
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    
    # Define que o campo "campos_faltantes" será uma lista de strings. O
    # Field (default_factory=list) garante que, se o modelo não retornar
    # nada para esse campo, ele será iniciado como uma lista vázia [], o
    # que evita erros no código. 
    # Observação: o default_factory irá definir o valor padrão da lista
    # (se o modelo não atribuir nenhum valor a lista).
    campos_faltantes: List[str] = Field(default_factory=list)
 

Vamos agora criar agora o objeto que utilizaremos para construir o agente de IA 

In [8]:
# Instância da classe
llm_triagem = ChatGoogleGenerativeAI(
    
    # Modelo que será utilizado
    model = 'gemini-2.5-flash',
    
    # Nivel de criativiade do modelo
    temperature = 0,
    
    # Chave que irá permitir a conexão com a API do google.
    api_key = api_key
)

Esse trecho irá definir a função "triagem", que usa o langchain para automatizar o processo de análise e classificação de uma mensagem. A função
é a ponte entre a mensagem do usuário e o modelo do Gemini.

In [9]:
from langchain_core.messages import SystemMessage, HumanMessage

# Ira criar a cadeia que faz duas coisas:
# chama o modelo: Ela usa a instância do modelo que criamos (llm_triagem).

# Garante o formato de saida: O método with_strNuctured_output(TriagemOut)
# instrui o modelo a retornar a resposta em um formato que corresponde
# exatamente a classe TriagemOut que você definiu com o Pydantic. Isso
# garante que a saida será um JSON válido.
triagem_chain = llm_triagem.with_structured_output(TriagemOut)

# Define a função triagem, que recebe como argumento a mensagem do usuário e retornara um dicionário
def triagem(mensagem: str) -> Dict:
    
    # Ira conter a função invoke que recebera como argumento:
    saida: TriagemOut = triagem_chain.invoke([
        
        # Define a instrução inicial para o modelo. Ele diz ao Gemini qual
        # papel ele deve assumir (triador de Service Desk) e quais regras
        # ele deve seguir.
        SystemMessage(content=TRIAGEM_PROMPT),
        
        # Ira conter a mensagem do usuário para o modelo.
        HumanMessage(content=mensagem)
        
    ]
    )
    
    # O pydantic retorna a resposta como um objeto TriagemOut. O método
    # model_dump() converte esse objeto para um dicionário padrão do Python,
    # o que facilita o uso posterior no seu código, como a leitura dos campos
    # "decisao", "urgencia", etc. 
    return saida.model_dump()
    


Vamos agora testar o modelo criando uma lista de perguntas para ele classificar as perguntas

In [10]:
testes = ["Posso reembolsar a internet?", "Quero mais dias de trabalho remoto, como faço?", "posso reembolsar cursos ou treinamentos da Alura?", "Quantas capivaras tem no rio pinheiros"]

In [11]:
# For que irá percorrer a lista de testes com o objetivo de acessar e passar
# para o modelo todas as peguntas da lista
for mensagem_texto in testes:
    
    # Ira imprimir a instrução do usuário (mensagem_texto que contém o conteudo do for) e a
    # resposta do modelo (classificação da instrução).
    print(f"Pergunta: {mensagem_texto}\n -> Resposta: {triagem(mensagem_texto)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais dias de trabalho remoto, como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no rio pinheiros
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}



            Aula 2: Construindo a base de conhecimento com RAG

In [17]:
from pathlib import Path

from langchain_community.document_loaders import PyMuPDFLoader 

docs = []

for ndocs in Path("pdfs").glob("*.pdf"):
    
    try:
        
        loader = PyMuPDFLoader(str(ndocs))
        
        docs.extend(loader.load())
        
        print(f"O arquivo {ndocs.name} foi carregado com sucesso")
    
    except Exception as erro:
        
        print(f"Erro ao carregar o arquivo {ndocs.name}: {erro}")
 
print(f"Total de documentos carregados: {len(docs)}")   
    

O arquivo Política de Reembolsos (Viagens e Despesas).pdf foi carregado com sucesso
O arquivo Política de Uso de E-mail e Segurança da Informação.pdf foi carregado com sucesso
O arquivo Políticas de Home Office.pdf foi carregado com sucesso
Total de documentos carregados: 3


In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

print(chunks)

[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='Política de Reembolsos (Viagens e \nDespesas) \n \n1.\u200b Reembolso: requer nota fiscal e deve ser submetido em até 10 dias corridos após a \ndespesa.\u200b\n \n2.\u200b Alimentação em viagem: limite de R$ 70/dia por pessoa. Bebidas alcoólicas não \nsão reembolsáveis.\u200b'), Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf

In [24]:
for chunk in chunks:
    
    print(chunk)
    print("------------------------------------------------")

page_content='Política de Reembolsos (Viagens e 
Despesas) 
 
1.​ Reembolso: requer nota fiscal e deve ser submetido em até 10 dias corridos após a 
despesa.​
 
2.​ Alimentação em viagem: limite de R$ 70/dia por pessoa. Bebidas alcoólicas não 
são reembolsáveis.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
------------------------------------------------
page_content='são reembolsáveis.​
 
3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. 
Comprovantes obrigatórios.​
 
4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, 
conf

                                    Embedding

-> Em Inteligência artificial, um embedding é uma forma de representar dados
(como palavras, frases, imagens, ou qualquer outra coisa) como uma sequência
de numeros, ou seja, um vetor.

-> Pense nisso como um mapa. Se você quer representar cidades em um mapa, você
usa duas coordenadas: latitude e longitude. Cidades que estão perto uma da outra no mundo real também estarão perto no mapa.

-> Com os embeddings, a lógica é a mesma, mas em um espaço com muita dimensões
(centenas ou até milhares). O que importa não éa posição, mas sim a distância 
entre os valores.

    -> Palavras ou frases com significado semelhante (ex: "cachorro", "cão", "pet") terão vetores com distâncias muito pequenas entre si.

    -> Palavras com significados opostos ou não relacionados ("cachorro", "carro", "nuvem") terão vetores com distâncias grandes.

-> O principal objetivo de um embedding é traduzir informações complexas para
uma linguagem numérica que os modelos de IA conseguem entender e processar de 
forma eficiente. 

In [27]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    
    model = "models/gemini-embedding-001",
    
    google_api_key = api_key
)

In [32]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

buscador = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold":0.3, "k": 4})

In [38]:
from langchain_core.prompts import  ChatPromptTemplate

from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),
    
      ("human", "Pergunta: {pergunta}\n\nContexto:\n{context}")
    
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [41]:
def perguntar_politica_rag(pergunta:str) -> Dict:
    
    docs_relacionados = buscador.invoke(pergunta)
    
    if not docs_relacionados:
        
        return {"answer": "Não sei", "citações":[], "contexto_encontrado": False}
    
    answer = document_chain.invoke({"pergunta":pergunta, "context": docs_relacionados })
    
    txt = (answer or "").strip()
    
    if txt.rstrip(".!?") == "Não sei":
        
        return {"answer": "Não sei", "citações":[], "contexto_encontrado": False}
    
    return {"answer": txt, "citações":docs_relacionados, "contexto_encontrado": True}

In [35]:
testes = ["Posso reembolsar a internet?", "Quero mais dias de trabalho remoto, como faço?", "posso reembolsar cursos ou treinamentos da Alura?", "Quantas capivaras tem no rio pinheiros"]

In [45]:
for i in testes:
    
    resposta = perguntar_politica_rag(i)
    
    print(f"pergunta: {i}")
    
    print(f"Resposta: {resposta['answer']}")
    
    if resposta['contexto_encontrado']:
        
        print(f"citações: {resposta['citações']}")
        
        print("------------------------------------")

pergunta: Posso reembolsar a internet?
Resposta: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
citações: [Document(id='df59fb00-1132-4ad5-b222-66c6df9ddc43', metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='são reembolsáveis.\u200b\n \n3.\u200b Transporte: táxi/app são permitidos quando não houver alternativa viável. \nComprovantes obrigatórios.\u200b\n \n4.\u200b Internet para home office: reembolsável via subsídio mensal de até R$ 100, \nconforme política de Home Office.\u200b'), Document(id='6fe7567c-d